In [2]:
import SimpleITK as sitk
import os
import matplotlib.pyplot as plt

In [3]:
def display_image(img):
    plt.imshow(sitk.GetArrayFromImage(img), cmap='gray')
    plt.axis("off")

def overlay_fixed_moving(fixed, moving):
    img1 = sitk.Cast(sitk.RescaleIntensity(fixed), sitk.sitkUInt8)
    img2 = sitk.Cast(sitk.RescaleIntensity(moving), sitk.sitkUInt8)
    cimg = sitk.Compose(img1, img2, sitk.Cast(img1/2. + img2/2., sitk.sitkUInt8))
    plt.imshow(sitk.GetArrayFromImage(cimg))
    plt.axis("off")

# Point-based Registation

Point-based registration allows us to help the registration via pre-defined sets of corresponding points. The `CorrespondingPointsEuclideanDistanceMetric` minimises the distance of between a points on the fixed image and corresponding points on the moving image. The metric can be used to help in a difficult registration task by taking into account positions are known to correspond. Think of it as a way of embedding expert knowledge in the registration procedure. We can manually select points or automatically them via centroids of segmentations for example. Anything is possible.

To use `CorrespondingPointsEuclideanDistanceMetric` we append it to the list of metrics in the parameter map.

In [4]:
parameterMap = sitk.GetDefaultParameterMap("bspline")
parameterMap["Metric"].append("CorrespondingPointsEuclideanDistanceMetric")

AttributeError: 'tuple' object has no attribute 'append'

---

#### Note

The `CorrespondingPointsEuclideanDistanceMetric` metric must be specified as the last metric due to technical constraints in elastix.

---

The metric can also be added to all metrics in the ElastixImageFilter object with a single call.

In [6]:
elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.AddParameter( "Metric", "CorrespondingPointsEuclideanDistanceMetric" )

<SimpleITK.SimpleITK.ElastixImageFilter; proxy of <Swig Object of type 'itk::simple::ElastixImageFilter::Self *' at 0x1203553f0> >

Or to a single parameter map (here we assume that SimpleElastix contains at least two parameter maps)

In [7]:
elastixImageFilter.AddParameter( 1, "Metric", "CorrespondingPointsEuclideanDistanceMetric" )

<SimpleITK.SimpleITK.ElastixImageFilter; proxy of <Swig Object of type 'itk::simple::ElastixImageFilter::Self *' at 0x1203557e0> >

#TODO: fix URL

The point set are specified as text files. They can either be in [VTK pointdata legacy format](http://dunne.uni-hd.de/VisuSimple/documents/vtkfileformat.html#pointdata) or elastix’ own format that usually has the `pts` extension.

```
<index, point>
<number of points>
point1 x point1 y [point1 z]
point2 x point2 y [point2 z]
```

`index` is used when points are specified in pixel coordinates. `point` is used when the points are specified in world coordinates. For example, this is a valid point file:

```
point
3
102.8 -33.4 57.0
178.1 -10.9 14.5
180.4 -18.1 78.9
```

---

#### Warning

The input points are specified in the fixed image domain (!) and warped from the fixed image to moving image since the transformation direction is from fixed to moving image. If we want to warp points from the moving image to fixed image, we need the inverse transform. This can be computed manually (see section 6.1.6 in the [elastix manual](http://elastix.isi.uu.nl/download/elastix_manual_v4.8.pdf)) or via `elastixImageFilter.ExecuteInverse()`.

---